In [37]:
import pandas as pd
import os

training_data = pd.read_csv("../input/iot-device-identification/iot_device_test.csv")
testing_data = pd.read_csv("../input/iot-device-identification/iot_device_train.csv")

In [38]:
training_data.head()

,ack,ack_A,ack_B,bytes,bytes_A,bytes_A_B_ratio,bytes_B,ds_field_A,ds_field_B,duration,...,suffix_is_co.il,suffix_is_com,suffix_is_com.sg,suffix_is_else,suffix_is_empty_char_value,suffix_is_googleapis.com,suffix_is_net,suffix_is_org,suffix_is_unresolved,device_category
0,38,20,18,14698,7814,1.135096,6884,0,0,1.9156,...,0,0,0,1,0,0,0,0,0,baby_monitor
1,38,20,18,14698,7814,1.135096,6884,0,0,2.0020,...,0,0,0,1,0,0,0,0,0,baby_monitor
2,38,20,18,14762,7814,1.124640,6948,0,0,2.1833,...,0,0,0,1,0,0,0,0,0,baby_monitor
3,38,20,18,14762,7814,1.124640,6948,0,0,2.1521,...,0,0,0,1,0,0,0,0,0,baby_monitor
4,41,20,21,14862,7994,1.163949,6868,0,0,2.2422,...,0,0,0,1,0,0,0,0,0,baby_monitor


In [39]:
training_data["device_category"].unique()

array(['baby_monitor', 'lights', 'motion_sensor', 'security_camera',
       'smoke_detector', 'socket', 'thermostat', 'TV', 'watch'],
      dtype=object)

In [40]:
testing_data["device_category"].unique()

array(['security_camera', 'TV', 'smoke_detector', 'thermostat',
       'water_sensor', 'watch', 'baby_monitor', 'motion_sensor', 'lights',
       'socket'], dtype=object)

In [41]:

for i in testing_data["device_category"].unique():
    if i not in training_data["device_category"].unique():
        print(i)

water_sensor


In [42]:

for i in training_data["device_category"].unique():
    if i not in testing_data["device_category"].unique():
        print(i)

In [43]:
len(training_data["device_category"].unique()), len(testing_data["device_category"].unique())

(9, 10)

lets drop the rows with water sensor as column

In [44]:
index_names = testing_data[testing_data['device_category'] == 'water_sensor'].index
len(index_names)

100

In [45]:
testing_data.drop(index_names, inplace=True)

In [46]:
testing_data["device_category"].unique()

array(['security_camera', 'TV', 'smoke_detector', 'thermostat', 'watch',
       'baby_monitor', 'motion_sensor', 'lights', 'socket'], dtype=object)

In [47]:

X_train, y_train = (
    training_data.loc[:, training_data.columns != "device_category"].values,
    training_data["device_category"],
)
X_test, y_test = (
    testing_data.loc[:, testing_data.columns != "device_category"].values,
    testing_data["device_category"],
)

In [48]:

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(training_data["device_category"].unique())
y_train_encoded = le.transform(y_train)
y_test_encoded = le.transform(y_test)


In [49]:
from xgboost import XGBClassifier

model = XGBClassifier()

In [50]:
model.fit(X_train, y_train_encoded)

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[09:48:31] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [51]:
model.score(X_test, y_test_encoded)

0.48